# Formatação de JSON

## Setup

### Imports

In [1]:
import json
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import nltk
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

c:\Users\flash\Documents\GitHub\AIService_Junipy\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\flash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\flash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\flash\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\flash\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Estabelecendo conexão com o chromadb

In [2]:
client = chromadb.PersistentClient(path="../../../docs/my_db")

### Criando/Carregando Collections

In [3]:
# nutrition = client.get_or_create_collection(name="nutrition")
comorbidity = client.get_or_create_collection(name="comorbidity")

## Json's

In [83]:
docs = "../../../docs/datasets/Dietary Guidelines for Americans.json"

In [95]:
docs = "../../../docs/datasets/KDOQI CLINICAL PRACTICE GUIDELINE FOR NUTRITION IN CKD 2020 UPDATE.json"

In [72]:
docs = "../../../docs/datasets/protocolo.json"

In [106]:
docs = "../../../docs/datasets/Management of Heart Failure.json"

In [60]:
docs = "../../../docs/datasets/Terapia Nutricional no Diabetes tipo 1.json"

In [117]:
docs = "../../../docs/datasets/Brazilian Guidelines of Hypertension – 2020.json"

In [4]:
docs = "../../../docs/datasets/Nutrição na insuficiencia renal.json"

## Normalização

In [5]:
df = pd.read_json(docs)

print(df)

      pagina   tipo                                           conteudo  \
0          1  texto       CLINICAL PRACTICE GUIDELINE FOR NUTRITION IN   
1          1  texto                CHRONIC KIDNEY DISEASE: 2019 UPDATE   
2          1  texto                                Public Review DRAFT   
3          1  texto                                       October 2019   
4          1  texto                                             Page 1   
...      ...    ...                                                ...   
7308     236  texto  512. Juraschek SP, Miller ER, 3rd, Weaver CM, ...   
7309     236  texto  Diet in Relation to Baseline Blood Pressure. J...   
7310     236  texto  513. Aaron KJ, Sanders PW. Role of dietary sal...   
7311     236  texto  disease: a review of the evidence. Mayo Clin P...   
7312     236  texto             Guideline on Nutrition in CKD Page 235   

      indice_tabela conteudo_dict conteudo_texto  
0               NaN           NaN            NaN  
1        

In [6]:
df = df[["conteudo"]]

df["conteudo"] = df["conteudo"].str.strip()
df = df[df["conteudo"].notnull() & (df["conteudo"] != "")]
df.to_json("../../../docs/datasets/df_normalizado.json", orient="records", force_ascii=False, indent=2)

print(df)


                                               conteudo
0          CLINICAL PRACTICE GUIDELINE FOR NUTRITION IN
1                   CHRONIC KIDNEY DISEASE: 2019 UPDATE
2                                   Public Review DRAFT
3                                          October 2019
4                                                Page 1
...                                                 ...
7308  512. Juraschek SP, Miller ER, 3rd, Weaver CM, ...
7309  Diet in Relation to Baseline Blood Pressure. J...
7310  513. Aaron KJ, Sanders PW. Role of dietary sal...
7311  disease: a review of the evidence. Mayo Clin P...
7312             Guideline on Nutrition in CKD Page 235

[7308 rows x 1 columns]


In [7]:
arquivo ="../../../docs/datasets/df_normalizado.json"

with open(arquivo, "r", encoding="utf-8") as f:
    dados = json.load(f)

df = pd.DataFrame(dados)
df.head

<bound method NDFrame.head of                                                conteudo
0          CLINICAL PRACTICE GUIDELINE FOR NUTRITION IN
1                   CHRONIC KIDNEY DISEASE: 2019 UPDATE
2                                   Public Review DRAFT
3                                          October 2019
4                                                Page 1
...                                                 ...
7303  512. Juraschek SP, Miller ER, 3rd, Weaver CM, ...
7304  Diet in Relation to Baseline Blood Pressure. J...
7305  513. Aaron KJ, Sanders PW. Role of dietary sal...
7306  disease: a review of the evidence. Mayo Clin P...
7307             Guideline on Nutrition in CKD Page 235

[7308 rows x 1 columns]>

In [8]:
chunks = df["conteudo"].tolist()

print("Chunks: ", len(chunks))
print(chunks[:5])

Chunks:  7308
['CLINICAL PRACTICE GUIDELINE FOR NUTRITION IN', 'CHRONIC KIDNEY DISEASE: 2019 UPDATE', 'Public Review DRAFT', 'October 2019', 'Page 1']


In [9]:
df_completo = " ".join(chunks)

print(df_completo[:500])

CLINICAL PRACTICE GUIDELINE FOR NUTRITION IN CHRONIC KIDNEY DISEASE: 2019 UPDATE Public Review DRAFT October 2019 Page 1 Guideline on Nutrition in CKD SECTION I: USE OF THE CLINICAL PRACTICE GUIDELINE This Clinical Practice Guideline document is based upon the best information available as of April 2017*. It is designed to provide information and assist decision making. It is not intended to define a standard of care, and should not be construed as one, nor should it be interpreted as prescribin


## Idioma json

In [78]:
api_stop_words = set(stopwords.words('portuguese')) 

In [12]:
api_stop_words = set(stopwords.words('english'))


## StopWords

In [13]:
def remove_stopwords(texto):
    palavras = texto.split()
    filtradas = [p for p in palavras if p.lower() not in api_stop_words]
    return ' '.join(filtradas)
    
chunks_limpos = [remove_stopwords(chunk) for chunk in chunks]
chunks_limpos = [chunk.lower() if isinstance(chunk, str) else chunk for chunk in chunks_limpos]

print("chunks limpos: ", len(chunks_limpos))
print(chunks_limpos[:50])


## Json em pt br = APENAS protocolo e terapia nutricional
## Todos os outros são em inglês
## Eu continuo isso amanhã


chunks limpos:  7308
['clinical practice guideline nutrition', 'chronic kidney disease: 2019 update', 'public review draft', 'october 2019', 'page 1', 'guideline nutrition ckd', 'section i: use clinical practice guideline', 'clinical practice guideline document based upon best information available', 'april 2017*. designed provide information assist decision making. intended', 'define standard care, construed one, interpreted', 'prescribing exclusive course management. variations practice inevitably', 'appropriately occur clinicians take account needs individual patients, available', 'resources, limitations unique institution type practice. every health-care', 'professional making use recommendations responsible evaluating', 'appropriateness applying setting particular clinical situation.', 'recommendations research contained within document general imply', 'specific protocol.', '*commissioned evidence review included articles published april 2017. consensus', 'opinion statements use l

## Lematização Portugues

In [ ]:

nlp = spacy.load("pt_core_news_sm")

def tratamento_pln(texto): 
    doc = nlp(texto) 
    tokens = [token.text for token in doc]
    clean_tokens = [token.lemma_ for token in doc if not token.is_punct] 
    clean_text = ' '.join(clean_tokens) 
    return clean_text

chunks_lemmatized = [tratamento_pln(chunk) for chunk in chunks_limpos]

print("chunks lemmatized: ", len(chunks_lemmatized))
print(chunks_lemmatized[:50])


## Lematização Ingles


In [14]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)  # Padrão: substantivo (NOUN)

# Função para lematizar um texto
def lemmatize_text(texto):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(texto)  # Tokenizar o texto
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]
    return ' '.join(lemmatized)

# Aplicar a lematização em chunks_limpos
chunks_lemmatized = [lemmatize_text(chunk) for chunk in chunks_limpos]

# Exibir os resultados
print("chunks lematizados: ", len(chunks_lemmatized))
print(chunks_lemmatized[:50])

chunks lematizados:  7308
['clinical practice guideline nutrition', 'chronic kidney disease : 2019 update', 'public review draft', 'october 2019', 'page 1', 'guideline nutrition ckd', 'section i : use clinical practice guideline', 'clinical practice guideline document base upon best information available', 'april 2017 * . design provide information assist decision make . intend', 'define standard care , construe one , interpret', 'prescribe exclusive course management . variation practice inevitably', 'appropriately occur clinician take account need individual patient , available', 'resource , limitation unique institution type practice . every health-care', 'professional make use recommendation responsible evaluate', 'appropriateness apply set particular clinical situation .', 'recommendation research contain within document general imply', 'specific protocol .', '* commission evidence review include article publish april 2017. consensus', 'opinion statement use literature publish tho

## Embedding e Tokenização

In [15]:
# Inicializar modelo
model = SentenceTransformer('all-MiniLM-L6-v2')  # rápido e de alta qualidade

# Escolher os chunks: lematizados ou apenas limpos
chunks_para_embedding = chunks_lemmatized  # ou chunks_limpos

# Gerar embeddings
embeddings = model.encode(chunks_para_embedding, batch_size=64, show_progress_bar=True)

print("Quantidade de embeddings geradas:", len(embeddings))
print("Exemplo de embedding:", embeddings[0][:10])  # mostra os 10 primeiros valores


Batches: 100%|██████████| 115/115 [00:20<00:00,  5.73it/s]


Quantidade de embeddings geradas: 7308
Exemplo de embedding: [-0.01210896  0.00561677 -0.03571162  0.06816469 -0.05810282  0.01495037
  0.00498431  0.089397   -0.09641775 -0.09039714]


## Salvar no Chromadb

In [16]:
import chromadb

client = chromadb.PersistentClient(path="../../../docs/my_db")
collection = client.get_or_create_collection(name="comorbidity")

ids = [f"doc_{i}" for i in range(len(embeddings))]
documents = chunks_para_embedding  # ou tokens

batch_size = 500
for i in range(0, len(embeddings), batch_size):
    batch_ids = ids[i:i+batch_size]
    batch_embeddings = embeddings[i:i+batch_size].tolist()
    batch_documents = documents[i:i+batch_size]

    collection.add(
        ids=batch_ids,
        embeddings=batch_embeddings,
        documents=batch_documents
    )

print(f"{len(embeddings)} embeddings adicionadas com sucesso!")


7308 embeddings adicionadas com sucesso!


## Querries de teste

In [ ]:
results = comorbidity.query(
    query_texts=[
        "Quais doenças costumam ocorrer junto com diabetes tipo 2?",
        "Qual comorbidade está mais associada à hipertensão?",
        "Pacientes com insuficiência renal costumam ter quais outras condições?",
        "Quais comorbidades aumentam o risco de complicações cardiovasculares?"
    ],
    n_results=5,
    # where={"origem": "tbca"}  # ou outro filtro que você tenha
)

for i, query_results in enumerate(results["documents"]):
    print(f"\nQuery {i}")
    print("\n".join(query_results))



Query 0
existe dieta padronizada pessoas diabetes tipo 1 , devendo
diretriz sociedade brasileira diabetes | 2
diretriz sociedade brasileira diabetes | 3
diretriz sociedade brasileira diabetes | 20
benefícios limitam tipo fibra , fonte tipo diabetes⁴⁹ .

Query 1
diferença significativa grupos acompanhamento . estudo apresentou
saúde especialmente acompanhamento individual potencializar demais
objetivos terapia nutricional são :
guia alimentar população brasileira .
análiser situação saúde plano ação estratégico enfrentamento

Query 2
tratamento destes pacientes . importante considerar possíveis benefícios
condições socioeconômicas , atividade física paciente , avaliar capacidade
qualquer indivíduo dm . paciente aprende identificar tamanho porções
1. thomopoulos c , parati g , zanchetti a. effect blood-pressure-lowering cardiovascular renal outcome : update systematic review meta-
2. packer m , anker sd , butler j , et al . cardiovascular renal outcome

Query 3
cardiovasculares³¹ . deve